In [13]:
%pip install PyMuPDF
%pip install PyPDF2
import re
import pandas as pd
import PyPDF2
import fitz

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
#ler pdf

def ler_pdf_para_dataframe(nome_arquivo):
    try:
        # Abrir o PDF
        pdf_document = fitz.open(nome_arquivo)
        dados = []

        # Ler todas as páginas e armazenar o texto em uma lista
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text = page.get_text("text")  # Extração de texto
            dados.append({"Página": page_num + 1, "Texto": text})

        # Fechar o documento
        pdf_document.close()
        
        # Criar DataFrame com os textos extraídos
        df = pd.DataFrame(dados)
        return df
    
    except Exception as e:
        print(f"Erro ao abrir o arquivo: {e}")
        return None

# Nome do arquivo PDF
nome_arquivo = "data/pdf/rotaer_completo.pdf"




In [15]:
#contar a quantidade de avezes de aparecimento dos termos

# Nome do arquivo PDF
nome_arquivo = "data/pdf/rotaer_completo.pdf"

def contar_termos_pdf(nome_arquivo):
    try:
        # Abrir o PDF
        pdf_document = fitz.open(nome_arquivo)
        texto_completo = ""

        # Ler todas as páginas e armazenar o texto
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            texto_completo += page.get_text("text") + "\n"  # Concatenar todo o texto

        # Fechar o documento
        pdf_document.close()

        # Contar as ocorrências dos termos no texto completo
        contagem_rdonav = len(re.findall(r'\bRDONAV\b', texto_completo, re.IGNORECASE))
        contagem_rffs = len(re.findall(r'\bRFFS\b', texto_completo, re.IGNORECASE))
        contagem_d_amdt = len(re.findall(r'\bD-AMDT\b', texto_completo, re.IGNORECASE))

        return {
            "RDONAV": contagem_rdonav,
            "RFFS": contagem_rffs,
            "D-AMDT": contagem_d_amdt
        }

    except Exception as e:
        print(f"Erro ao processar o arquivo '{nome_arquivo}': {e}")
        return None

# Contar os termos no PDF
contagem_termos = contar_termos_pdf(nome_arquivo)

# Exibir os resultados
print(f"Ocorrências de 'RDONAV': {contagem_termos['RDONAV']}")
print(f"Ocorrências de 'RFFS': {contagem_termos['RFFS']}")
print(f"Ocorrências de 'D-AMDT': {contagem_termos['D-AMDT']}")



Ocorrências de 'RDONAV': 144
Ocorrências de 'RFFS': 174
Ocorrências de 'D-AMDT': 6273


In [16]:
#CRIAR BASE DE CODIGO OACI+ INFO ROTAER 

def extrair_dados_pdf(nome_arquivo):
    try:
        pdf_document = fitz.open(nome_arquivo)
        texto_completo = []

        # Ler todas as páginas e armazenar o texto
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text = page.get_text("text")
            texto_completo.extend(text.split("\n"))

        pdf_document.close()

        dados_aeroportos = []
        capturando = False
        aeroporto_info = []
        codigo_oaci = None

        # Regex que procura o OACI entre parênteses com espaço antes e depois
        regex_oaci = r"\(\s*([S][A-Z0-9]{3})\s*\)"

        linhas_espera_oaci = 0  # Contador de linhas após o D-AMDT
        linha_damdt_num = 0     # Para registrar onde foi o último D-AMDT (para o alerta)

        idx = 0
        while idx < len(texto_completo):
            linha = texto_completo[idx].strip()

            # Detecta o início de um novo aeroporto
            if linha.startswith("D-AMDT"):
                # Salva o bloco anterior, se existir
                if codigo_oaci and aeroporto_info:
                    dados_aeroportos.append({
                        "codigo_aeroporto": codigo_oaci,
                        "infos_rotaer": "\n".join(aeroporto_info)
                    })

                capturando = True
                aeroporto_info = []
                codigo_oaci = None
                linhas_espera_oaci = 8  # Vamos procurar o OACI nas próximas 8 linhas
                linha_damdt_num = idx + 1  # Guardar o número da linha (contagem humana começa em 1)

            elif capturando and not codigo_oaci and linhas_espera_oaci > 0:
                # Preparar linha atual + próxima para capturar quebras de linha no título
                linha_atual = linha
                linha_proxima = texto_completo[idx + 1].strip() if (idx + 1) < len(texto_completo) else ""
                linha_combinada = linha_atual + " " + linha_proxima

                match = re.search(regex_oaci, linha_combinada)
                if match:
                    codigo_oaci = match.group(1)
                    # print(f"[INFO] OACI encontrado: {codigo_oaci} nas linhas {idx+1} e {idx+2}")

                linhas_espera_oaci -= 1

                if linhas_espera_oaci == 0 and not codigo_oaci:
                    print(f"[ALERTA] Não encontrou OACI no bloco iniciado com D-AMDT na linha {linha_damdt_num}.")

            elif capturando:
                # Apenas armazena o texto no bloco de informações
                aeroporto_info.append(linha)

            idx += 1  # Avança para a próxima linha

        # Salvar o último bloco se tiver informações
        if codigo_oaci and aeroporto_info:
            dados_aeroportos.append({
                "codigo_aeroporto": codigo_oaci,
                "infos_rotaer": "\n".join(aeroporto_info)
            })

        df_aeroportos = pd.DataFrame(dados_aeroportos)
        return df_aeroportos

    except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")
        return None

# Nome do arquivo PDF
nome_arquivo = "data/pdf/rotaer_completo.pdf"

df_aeroportos2 = extrair_dados_pdf(nome_arquivo)

# Mostrar um resumo dos códigos encontrados
if df_aeroportos2 is not None:
    print("\nResumo dos códigos capturados:")
    print(df_aeroportos2['codigo_aeroporto'].value_counts())

    # Para validar rapidamente
    if "SBGR" in df_aeroportos2['codigo_aeroporto'].values:
        print("\n✅ SBGR foi capturado com sucesso!")
    else:
        print("\n❌ SBGR ainda não foi encontrado.")



Resumo dos códigos capturados:
codigo_aeroporto
SJOQ    2
SJUR    2
SWGN    2
SDOX    2
SN9Z    2
       ..
SSZD    1
SJI9    1
SNBC    1
SD9P    1
SS7W    1
Name: count, Length: 5675, dtype: int64

✅ SBGR foi capturado com sucesso!


In [17]:
#Criar df apenas com os aeroportos que possuem RDONAV ou RFFS

# Filtrar as linhas onde a coluna 'infos_rotaer' contém "RDONAV" ou "RFFS"
aero_base = df_aeroportos2[
    df_aeroportos2['infos_rotaer'].str.contains(r'\b(RDONAV|RFFS)\b', flags=re.IGNORECASE, na=False)
]


C:\Users\julia\AppData\Local\Temp\ipykernel_11528\415894066.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_aeroportos2['infos_rotaer'].str.contains(r'\b(RDONAV|RFFS)\b', flags=re.IGNORECASE, na=False)


In [18]:
# Ordenar opcionalmente para garantir consistência (ex: ordem alfabética)
aero_base = aero_base.sort_values(by='codigo_aeroporto')

# Remover duplicatas com base no código OACI, mantendo apenas o primeiro
aero_base_semduplicado = aero_base.drop_duplicates(subset='codigo_aeroporto', keep='first')

# Exibir o DataFrame final sem duplicações
print(aero_base_semduplicado)



     codigo_aeroporto                                       infos_rotaer
435              SBAE  22 09 28S/049 04 06W\nAD PUB REDE VOA 18SE UTC...
599              SBAF  Campo Délio Jardim de Mattos ( SBAF ) / RIO DE...
607              SBAN  Campo Marechal Márcio de Souza e Mello ( SBAN ...
5654             SBAR  Santa Maria ( SBAR ) / ARACAJU, SE\n10 59 07S/...
5228             SBAT  Piloto Osvaldo Marques Dias ( SBAT ) / ALTA\nF...
...               ...                                                ...
4660             SWLB  07 16 44S/064 46 10W\nAD PUB 4SE UTC-5 HJ\n74 ...
5154             SWPI  PARINTINS ( SWPI ) / PARINTINS, AM\n02 40 25S/...
5229             SWPM  11 38 29S/061 10 44W\nAD PUB 4S UTC-4 VFR L21 ...
5937             SWTS  TANGARA DA SERRA ( SWTS ) / TANGARÁ DA SERRA,\...
531              SWUZ  LUZIÂNIA, GO\n16 15 42S/047 58 07W\nAD PUB INF...

[146 rows x 2 columns]


In [19]:
#Extrair os dados de Ctegoria Contra Incendio a partir do filtro do termo "RFFS"

def extrair_rffs(texto):
    """
    Função para extrair corretamente 'RFFS' seguido de:
    - 'CAT CIVIL - número' (sempre)
    - '[X] CAT CIVIL - número' (se houver número antes entre colchetes)
    - 'CAT MILITAR - número' (se existir)
    - Mantém apenas essas informações, removendo qualquer coisa extra
    """
    if pd.isna(texto):  # Se o valor for NaN, retorna None
        return None

    # Expressão regular aprimorada para capturar o necessário
    match = re.search(r'RFFS\s*([\[\]\w\s-]*\d+)', texto, re.IGNORECASE)  # Agora inclui colchetes `[]`

    if match:
        return match.group(1).strip()  # Retorna apenas o conteúdo desejado

    return None  # Retorna None se não encontrar nada válido

# Criar nova base `aero_base_rffs2`
aero_base_rffs = aero_base_semduplicado.copy()
aero_base_rffs['RFFS'] = aero_base_rffs['infos_rotaer'].apply(extrair_rffs)

# Exibir os primeiros resultados para conferência
print(aero_base_rffs[['codigo_aeroporto', 'RFFS']])

     codigo_aeroporto             RFFS
435              SBAE             None
599              SBAF    - CAT MIL - 6
607              SBAN    - CAT MIL - 6
5654             SBAR  - CAT CIVIL - 7
5228             SBAT             None
...               ...              ...
4660             SWLB             None
5154             SWPI  - CAT CIVIL - 5
5229             SWPM  - CAT CIVIL - 5
5937             SWTS             None
531              SWUZ             None

[146 rows x 2 columns]


In [20]:
# Listas de auxílios
auxilios_radio = ["VOR", "DME", "VORTAC", "TACAN", "NDB", "LMM", "LOM", "ILS", "LOC", "GS", "IM", "MM", "OM"]
pares_auxilios = ["ILS/DME", "VOR/DME", "LOC/DME"]

def extrair_auxilios_por_etapa(texto):
    texto = texto.upper()
    texto = texto.replace('\n', ' ').replace('\xa0', ' ')
    texto = re.sub(r'\s+', ' ', texto)

    # Normalização de pares quebrados
    texto = re.sub(r"I\s*L\s*S\s*/\s*D\s*M\s*E", "ILS/DME", texto)
    texto = re.sub(r"V\s*O\s*R\s*/\s*D\s*M\s*E", "VOR/DME", texto)
    texto = re.sub(r"L\s*O\s*C\s*/\s*D\s*M\s*E", "LOC/DME", texto)

    pos_rdonav = texto.find("RDONAV")
    if pos_rdonav == -1:
        return None, None

    # Limitar a leitura a 100 caracteres após "RDONAV"
    trecho = texto[pos_rdonav:pos_rdonav + 300]
    pares_detectados = []
    usados = set()

    # Etapa 1 – detectar pares exatos e registrar apenas o par como 'usado'
    for par in pares_auxilios:
        pattern = r'(?<![A-Z])' + re.escape(par) + r'(?![A-Z])'
        matches = list(re.finditer(pattern, trecho))
        for match in matches:
            pares_detectados.append(match.group())
            usados.add(par)  # Adiciona apenas o par completo
        trecho = re.sub(pattern, 'xxx', trecho)

    # Etapa 2 – detectar auxílios isolados mesmo que tenham sido parte de um par
    auxilios_individuais = []
    for aux in auxilios_radio:
        pattern = r'(?<![A-Z])' + re.escape(aux) + r'(?![A-Z])'
        if re.search(pattern, trecho):
            auxilios_individuais.append(aux)
            trecho = re.sub(pattern, 'xxx', trecho)

    return "\n".join(pares_detectados) if pares_detectados else None, \
           "\n".join(auxilios_individuais) if auxilios_individuais else None

# Aplicar a função nas colunas
aero_base_rffs[["Auxilios_Pares", "Auxilios_Individuais"]] = aero_base_rffs["infos_rotaer"].apply(
    lambda x: pd.Series(extrair_auxilios_por_etapa(x))
)


In [21]:
# Criar novo DataFrame final chamado 'base_rotaer_final'
base_rotaer = aero_base_rffs.copy()

# Salvar o DataFrame final em CSV
base_rotaer.to_csv("data/csv/base_rotaer.csv", index=False, encoding="utf-8")


In [22]:
# Função para remover linhas duplicadas (de RDONAV) mantendo a ordem
def limpar_auxilios(celula):
    if pd.isna(celula):
        return None
    linhas = celula.strip().split('\n')
    vistos = set()
    resultado = []
    for linha in linhas:
        if linha not in vistos:
            resultado.append(linha)
            vistos.add(linha)
    return '\n'.join(resultado)

# Aplica a limpeza nas colunas
base_rotaer["Auxilios_Pares"] = base_rotaer["Auxilios_Pares"].apply(limpar_auxilios)
base_rotaer["Auxilios_Individuais"] = base_rotaer["Auxilios_Individuais"].apply(limpar_auxilios)


In [23]:

# ----------- Função 1: Limpeza e extração dos padrões CAT -----------

def limpar_rffs(texto):
    """
    Extrai apenas:
    - CAT CIVIL - (1 a 10)
    - CAT MILITAR - (1 a 10)
    - CAT MIL (1 a 10)
    - CAT - (1 a 10)
    """
    if pd.isna(texto):
        return None

    # Regex corrigida para incluir 10 corretamente
    match = re.findall(
        r'(CAT\s*CIVIL\s*-\s*(?:10|[1-9])|CAT\s*MILITAR\s*-\s*(?:10|[1-9])|CAT\s*MIL\s*(?:10|[1-9])|CAT\s*-\s*(?:10|[1-9]))',
        texto,
        flags=re.IGNORECASE
    )

    if match:
        return " / ".join(match)
    else:
        return texto  # Retorna o original se nada for extraído

# ----------- Função 2: Padronização dos termos -----------

def padronizar_rffs(texto):
    """
    Padroniza os nomes:
    - "CAT - n" → "CAT CIVIL n"
    - "CAT MIL n" → "CAT MILITAR n"
    """
    if pd.isna(texto):
        return None

    # "CAT - 10" ou "CAT - 1" → "CAT CIVIL 10" ou "CAT CIVIL 1"
    texto = re.sub(r'CAT\s*-\s*(10|[1-9])', r'CAT CIVIL \1', texto, flags=re.IGNORECASE)

    # "CAT MIL 10" ou "CAT MIL 1" → "CAT MILITAR 10"
    texto = re.sub(r'CAT\s*MIL\s*(10|[1-9])', r'CAT MILITAR \1', texto, flags=re.IGNORECASE)

    return texto

# ----------- Aplicando ao DataFrame -----------

# Aplicar função de limpeza
base_rotaer['RFFS'] = base_rotaer['RFFS'].apply(limpar_rffs)

# Aplicar padronização
base_rotaer['RFFS'] = base_rotaer['RFFS'].apply(padronizar_rffs)

base_rotaer.to_csv("output/base_rotaer_final.csv", index=False, encoding="utf-8")